In [ ]:
pip install -q pytorch-adapt

In [1]:
import sys

sys.path.insert(0, "../../src")
import torch
import torchvision.transforms as T
import tqdm
from torchvision.datasets import MNIST

import pytorch_adapt
from pytorch_adapt.datasets.getters import get_mnist_mnistm
from pytorch_adapt.hooks import DANNHook
from pytorch_adapt.models import Classifier, Discriminator, MNISTFeatures
from pytorch_adapt.utils.common_functions import batch_to_device
from pytorch_adapt.utils.constants import IMAGENET_MEAN, IMAGENET_STD
from pytorch_adapt.utils.transforms import GrayscaleToRGB

print(pytorch_adapt.__version__)

0.0.11


In [6]:
datasets = get_mnist_mnistm(["mnist"], ["mnistm"], ".")
dataloader = torch.utils.data.DataLoader(
    datasets["train"], batch_size=32, num_workers=2
)

In [7]:
device = torch.device("cuda")

G = MNISTFeatures().to(device)
C = Classifier(num_classes=10, in_size=1200, h=256).to(device)
D = Discriminator(in_size=1200, h=256).to(device)
models = {"G": G, "C": C, "D": D}

G_opt = torch.optim.Adam(G.parameters(), lr=0.0001)
C_opt = torch.optim.Adam(C.parameters(), lr=0.0001)
D_opt = torch.optim.Adam(D.parameters(), lr=0.0001)
opts = [G_opt, C_opt, D_opt]

hook = DANNHook(opts)

In [8]:
for i, data in enumerate(tqdm.tqdm(dataloader)):
    data = batch_to_device(data, device)
    loss, loss_components = hook({}, {**models, **data})

c:\users\kevin\anaconda3\envs\pydocs\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


{'total_loss': {'src_domain_loss': 0.6505722999572754, 'target_domain_loss': 0.7259259223937988, 'c_loss': 2.3411178588867188, 'total': 1.2392053604125977}}
